# Useful Operations in TensorFlow

Adapted from [A Sweeping Tour of TensorFlow](http://www.goldsborough.me/tensorflow/ml/ai/python/2017/06/28/20-21-45-a_sweeping_tour_of_tensorflow/)
by Peter Goldsborough

In [21]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # No more warning
import tensorflow as tf

## If-Statements  

**tf.cond(
    pred,
    true_fn=None,
    false_fn=None,
    strict=False,
    name=None,
)**

The following document was adapted from  [TF documentation](https://www.tensorflow.org/api_docs/python/tf/cond) of this operator. 

This operation returns true_fn() if the predicate pred is true else false_fn(). 

In [2]:
x = tf.constant(3)
y = tf.constant(4)
z = tf.multiply(x, y)
result = tf.cond(x < y, lambda: tf.add(x, z), lambda: tf.square(y))

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
with tf.Session() as session:
    print(session.run(result, feed_dict={x: 9, y:7}))
    print(session.run(result, feed_dict={x: 7, y:9}))

49
70


In [4]:
def f1(): 
    return tf.multiply(x, 17)
def f2(): 
    return tf.add(y, 23)
result = tf.cond(tf.less(x, y), f1, f2)

In [5]:
with tf.Session() as session:
    print(session.run(result, feed_dict={x: 9, y:7}))
    print(session.run(result, feed_dict={x: 7, y:9}))

30
119


In [6]:
a = tf.constant([1, 2, 3])
b = tf.constant([4, 5, 6])
x = tf.placeholder(shape=[], dtype=tf.bool)

c = tf.cond(x, lambda: a, lambda: b)

with tf.Session() as session:
    print(session.run(c, feed_dict={x: True}))
    print(session.run(c, feed_dict={x: False}))

[1 2 3]
[4 5 6]


## While Loops 
**tf.while_loop(
    cond,
    body,
    loop_vars,
    shape_invariants=None,
    parallel_iterations=10,
    back_prop=True,
    swap_memory=False,
    name=None,
    maximum_iterations=None,
    return_same_structure=False
)**  

The following document was adapted from  [TF documentation](https://www.tensorflow.org/api_docs/python/tf/while_loop) of this operator. 

**tf.while_loop repeat body while the condition cond is true.** cond is a **callable** returning a boolean scalar tensor. body is a **callable** returning a (possibly nested) tuple, namedtuple or list of tensors of the same arity (length and structure) and types as loop_vars. loop_vars is a (possibly nested) tuple, namedtuple or list of tensors that is passed to both cond and body. cond and body both take as many arguments as there are loop_vars.  

**Returns:**  

The output tensors for the loop variables after the loop. If return_same_structure is True, the return value has the same structure as loop_vars. If return_same_structure is False, the return value is a Tensor, TensorArray or IndexedSlice if the length of loop_vars is 1, or a list otherwise.

In [7]:
i = tf.constant(1)
cond = lambda i: tf.less(i, 10)
body = lambda i: tf.multiply(i, 2)
r = tf.while_loop(cond, body, [i])
with tf.Session():
    print(r.eval())

16


In [8]:
i = tf.constant(1)
var_new = tf.while_loop(lambda i: i < 20, lambda i: i + 2, [i])
with tf.Session():
    print(var_new.eval())

21


In [9]:
i0 = tf.constant(0)
m0 = tf.constant([[1,2], [3,4]])
cond = lambda i, m: i < 10
body = lambda i, m: [i+1, tf.concat([m, m], axis=0)]
matrix =tf.while_loop(cond, body, loop_vars=[i0, m0],
    shape_invariants=[i0.get_shape(), tf.TensorShape([None, 2])])
with tf.Session() as sess:
    print(sess.run(matrix))

[10, array([[1, 2],
       [3, 4],
       [1, 2],
       ...,
       [3, 4],
       [1, 2],
       [3, 4]], dtype=int32)]


## Convert a Function of Python to TensorFlow Operation (tf.py_func)

**tf.py_func(
    func,
    inp,
    Tout,
    stateful=True,
    name=None
)**

tf.py_func wraps a python function and uses it as a TensorFlow op. The following document was adapted from  [TF documentation](https://docs.w3cub.com/tensorflow~python/tf/py_func/) of this operator. **Note that the tf.py_func operation will only run on CPU.** 

Given a python function func, which takes numpy arrays as its arguments and returns numpy arrays as its outputs, wrap this function as an operation in a TensorFlow graph. The following snippet constructs a simple TensorFlow graph that invokes the np.sinh() NumPy function as a operation in the graph:

In [10]:
def my_func1(x):
  return 2**x
input_1 = tf.placeholder(tf.int64)
op_1= tf.py_function(my_func1, [input_1], tf.int64)

In [11]:
with tf.Session() as sess:
    print(sess.run(op_1, feed_dict={input_1:5}))

32


In [12]:
def my_func2(x):
  # x will be a numpy array with the contents of the placeholder below
  return np.sinh(x)
input= tf.placeholder(tf.float32)
op_2= tf.py_func(my_func2, [input], tf.float32)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [13]:
import numpy as np
with tf.Session() as sess:
    print(sess.run(op_2, feed_dict={input:88}))

8.2581813e+37


## Variable Scopes

Variable scopes allow you to control variable reuse when calling functions which implicitly create and use variables. They also allow you to name your variables in a hierarchical and understandable way. The following document was adapted from  [TF documentation](https://www.tensorflow.org/api_docs/python/tf/variable_scope) of this operator.

Simple example of how to create a new variable:

In [14]:
with tf.variable_scope("big"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.name == "big/bar/v:0"

In [15]:
v

<tf.Variable 'big/bar/v:0' shape=(1,) dtype=float32_ref>

**Basic example of sharing a variable AUTO_REUSE:**  

In [16]:
# AUTO_REUSE: we create variables if they do not exist, and return them otherwise; if None, we inherit 
# the parent scope's reuse flag.
def make_var():
    with tf.variable_scope("make_var", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("new_v", dtype=tf.int32, initializer=tf.constant([1,2]))
    return v

v1=make_var()  # Creates v
v2=make_var()  # Gets the same, existing v
assert v1 == v2

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v1))
    print(sess.run(v2))

[1 2]
[1 2]


Basic example of sharing a variable with reuse=True:

In [18]:
with tf.variable_scope("foo"):
    v = tf.get_variable("my_vec", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("my_vec", [1])
assert v1 == v

To prevent accidental sharing of variables, we raise an exception when getting an existing variable in a non-reusing scope.

In [19]:
# #  Raises ValueError("... v already exists ...").
# with tf.variable_scope("foo"):
#     v = tf.get_variable("v", [1])
#     v1= tf.get_variable("v", [1])

Similarly, we raise an exception when trying to get a variable that does not exist in reuse mode.

In [20]:
# #  Raises ValueError("... v does not exists ...").
# with tf.variable_scope("foo", reuse=True):
#     v = tf.get_variable("new_variable", [1])